# Import Libraries

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from typing import Dict, Optional
from tqdm import tqdm

# Define Functions

In [2]:
def get_tickers(url):
    """Fetches base data from a given URL and returns it as a DataFrame.
    Args:
        url (str): The URL to fetch the data from.
    Returns:
        pd.DataFrame: A DataFrame containing the fetched data.
    """
    
    df = pd.read_html(url)[0]
    df = df.rename(columns={'Symbol': 'Ticker', 'Security': 'Company_Name', 'GICS Sector': 'Sector', 'GICS Sub-Industry': 'Industry', 'Founded': 'Founded_Year'}).drop(['Date added', 'CIK'], axis=1)
    df.columns = df.columns.str.replace(' ', '_').str.replace('/', '_').str.replace('-', '_')
    df['Ticker'] = df['Ticker'].str.upper()
    
    print(f"Fetched {len(df)} rows from {url}")
    return df['Ticker'].unique().tolist()

In [15]:


def get_stock_data(ticker_symbol: str, period: str = "5y", benchmark_data: Optional[pd.DataFrame] = None) -> Dict:
    """
    Calculate comprehensive stock metrics for a given ticker.
    
    Args:
        ticker_symbol (str): Stock ticker symbol (e.g., 'AAPL')
        period (str): Historical data period (default: '5y')
        benchmark_data (pd.DataFrame): Pre-loaded S&P 500 data for beta calculation
        
    Returns:
        Dict: Dictionary containing all calculated metrics for the ticker
    """
    try:
        ticker = yf.Ticker(ticker_symbol)
        
        # Get historical data
        hist = ticker.history(period=period)
        if hist.empty:
            raise ValueError(f"No data found for {ticker_symbol}")
        
        hist = hist.reset_index()
        hist['Date'] = pd.to_datetime(hist['Date'])
        hist['Year'] = hist['Date'].dt.year
        
        # Get ticker info
        info = ticker.info
        current_year = hist['Year'].max()
        
        # Initialize results dictionary
        stock_data = {'Ticker': ticker_symbol}
        
        # Price-based metrics
        stock_data.update(_calculate_price_metrics(hist))
        
        # Return calculations
        stock_data.update(_calculate_returns(hist, current_year))
        
        # Risk metrics (pass benchmark_data to avoid reloading)
        stock_data.update(_calculate_risk_metrics(hist, benchmark_data))
        
        # Market data
        stock_data.update(_get_market_data(info))
        
        return stock_data
        
    except Exception as e:
        print(f"Error processing {ticker_symbol}: {e}")

def _calculate_price_metrics(hist: pd.DataFrame) -> Dict:
    """Calculate price-related metrics."""
    closing_price = hist['Close'].iloc[-1]
    all_time_high = hist['Close'].max()
    
    # Calculate moving average
    hist['200_Day_Moving_Average'] = hist['Close'].rolling(window=200).mean()
    
    return {
        'Closing_Price': round(closing_price, 2),
        'All_Time_High': round(all_time_high, 2),
        'Percent_From_All_Time_High': round(((closing_price - all_time_high) / all_time_high) * 100, 2),
        'Percent_Difference_200_Day_Moving_Average': round(((closing_price - hist['200_Day_Moving_Average'].iloc[-1]) / hist['200_Day_Moving_Average'].iloc[-1]) * 100, 2),
        '24_Hour_Percent_Change': round(hist['Close'].pct_change(periods=1).iloc[-1] * 100, 2),
        '7_Day_Percent_Change': round(hist['Close'].pct_change(periods=7).iloc[-1] * 100, 2),
        '30_Day_Percent_Change': round(hist['Close'].pct_change(periods=30).iloc[-1] * 100, 2)
    }


def _calculate_returns(hist: pd.DataFrame, current_year: int) -> Dict:
    """Calculate return metrics."""
    returns_data = {}
    
    # Annualized return
    total_years = len(hist['Year'].unique())
    if total_years > 1:
        annualized_return = ((hist['Close'].iloc[-1] / hist['Close'].iloc[0]) ** (1 / total_years) - 1) * 100
        returns_data['Annualized_Return'] = round(annualized_return, 2)
    
    # Yearly returns
    for year in sorted(hist['Year'].unique(), reverse=True):
        year_data = hist[hist['Year'] == year]
        if len(year_data) > 1:
            year_return = ((year_data['Close'].iloc[-1] - year_data['Close'].iloc[0]) / year_data['Close'].iloc[0]) * 100
            
            if year == current_year:
                returns_data['YTD_Return'] = round(year_return, 2)
            else:
                returns_data[f'{year}_Return'] = round(year_return, 2)
    
    return returns_data


def _calculate_risk_metrics(hist: pd.DataFrame, benchmark_data: Optional[pd.DataFrame] = None) -> Dict:
    """Calculate risk-related metrics."""
    # Daily returns
    hist['Daily_Return'] = hist['Close'].pct_change().dropna()
    
    # Volatility (annualized)
    volatility = hist['Daily_Return'].std() * np.sqrt(252)
    
    # Sharpe ratio (assuming 1% risk-free rate)
    risk_free_rate = 0.01
    if len(hist) > 1:
        total_return = (hist['Close'].iloc[-1] / hist['Close'].iloc[0]) - 1
        annualized_return = (1 + total_return) ** (252 / len(hist)) - 1
        excess_return = annualized_return - risk_free_rate
        sharpe_ratio = excess_return / volatility if volatility > 0 else 0
    else:
        sharpe_ratio = 0
    
    # Beta calculation (use pre-loaded benchmark data if available)
    beta = _calculate_beta(hist, benchmark_data)
    
    return {
        'Annualized_Volatility': round(volatility * 100, 2),
        'Sharpe_Ratio': round(sharpe_ratio, 2),
        'Beta': round(beta, 2) if not np.isnan(beta) else None
    }


def _calculate_beta(hist: pd.DataFrame, benchmark_data: Optional[pd.DataFrame] = None) -> float:
    """Calculate beta against S&P 500 using pre-loaded benchmark data."""
    try:
        if benchmark_data is None:
            return np.nan
            
        # Merge on date
        merged = hist.merge(
            benchmark_data[['Date', 'Daily_Return']], 
            on='Date', 
            suffixes=('', '_Benchmark'),
            how='inner'
        )
        
        if len(merged) < 30:  # Need sufficient data points
            return np.nan
        
        # Calculate beta using numpy for speed
        stock_returns = merged['Daily_Return'].dropna()
        benchmark_returns = merged['Daily_Return_Benchmark'].dropna()
        
        if len(stock_returns) == len(benchmark_returns) and len(stock_returns) > 0:
            covariance = np.cov(stock_returns, benchmark_returns)[0, 1]
            benchmark_variance = np.var(benchmark_returns)
            return covariance / benchmark_variance if benchmark_variance != 0 else np.nan
        
        return np.nan
        
    except Exception:
        return np.nan


def _load_benchmark_data(period: str = "5y") -> pd.DataFrame:
    """Load S&P 500 benchmark data once for all calculations."""
    try:
        print("Loading S&P 500 benchmark data...")
        benchmark = yf.Ticker('^GSPC')
        benchmark_hist = benchmark.history(period=period).reset_index()
        benchmark_hist['Date'] = pd.to_datetime(benchmark_hist['Date'])
        benchmark_hist['Daily_Return'] = benchmark_hist['Close'].pct_change()
        return benchmark_hist[['Date', 'Daily_Return']].dropna()
    except Exception as e:
        print(f"Failed to load benchmark data: {e}")
        return pd.DataFrame()


def _get_market_data(info: Dict) -> Dict:
    """Extract market data from ticker info."""
    return {
        'Company_Name': info.get('shortName', '').replace('\n', ' '),
        'Market_Cap': info.get('marketCap'),
        'Sector': info.get('sector'),
        'Industry': info.get('industry'),
        'Country': info.get('country'),
        'Business_Summary': info.get('longBusinessSummary', '').replace('\n', ' '),
        'Dividend_Yield': info.get('dividendYield'),
        'Trailing_PE': info.get('trailingPE'),
        'Forward_PE': info.get('forwardPE'),
        'Average_Volume': info.get('averageVolume'),
        'Average_Volume_10days': info.get('averageVolume10days'),
        '52_Week_Change': info.get('52WeekChange')
    }


def get_multiple_stocks_data(tickers: list, period: str = "5y") -> pd.DataFrame:
    """
    Get stock data for multiple tickers and return as DataFrame.
    
    Args:
        tickers (list): List of ticker symbols
        period (str): Historical data period
        
    Returns:
        pd.DataFrame: DataFrame with stock data for all tickers
    """
    # Pre-load benchmark data once for all stocks
    benchmark_data = _load_benchmark_data(period)
    
    all_data = []
    
    # Process stocks sequentially with progress bar
    for ticker in tqdm(tickers, desc="Processing stocks", unit="stock"):
        stock_data = get_stock_data(ticker, period, benchmark_data)
        all_data.append(stock_data)
    
    df = pd.DataFrame(all_data)
    return df.sort_values('Market_Cap', ascending=False, na_position='last')

# Get List of Tickers

In [19]:
stock_tickers = get_tickers('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[:50]

Fetched 503 rows from https://en.wikipedia.org/wiki/List_of_S%26P_500_companies


# Create Dataset with Features

In [20]:
df_enriched_stock_data = get_multiple_stocks_data(stock_tickers)
df_enriched_stock_data

Loading S&P 500 benchmark data...


Processing stocks: 100%|██████████| 50/50 [01:59<00:00,  2.40s/stock]


,Ticker,Closing_Price,All_Time_High,Percent_From_All_Time_High,Percent_Difference_200_Day_Moving_Average,24_Hour_Percent_Change,7_Day_Percent_Change,30_Day_Percent_Change,Annualized_Return,YTD_Return,...,Sector,Industry,Country,Business_Summary,Dividend_Yield,Trailing_PE,Forward_PE,Average_Volume,Average_Volume_10days,52_Week_Change
39,AAPL,196.58,258.40,-23.92,-12.05,0.48,-2.42,-0.84,15.00,-19.19,...,Technology,Consumer Electronics,United States,"Apple Inc. designs, manufactures, and markets ...",0.53,30.572319,23.655836,61130764,51288240,-0.052581
22,AMZN,212.52,242.06,-12.20,4.29,-1.07,-2.06,14.87,8.02,-3.50,...,Consumer Cyclical,Internet Retail,United States,"Amazon.com, Inc. engages in the retail sale of...",NaN,34.612380,34.556100,48685895,36713730,0.123969
20,GOOG,173.98,207.22,-16.04,0.57,-1.83,-2.05,5.44,16.07,-8.52,...,Communication Services,Internet Content & Information,United States,Alphabet Inc. offers various products and plat...,0.48,19.417410,19.439106,26395583,22489300,-0.034839
19,GOOGL,173.32,205.89,-15.82,1.13,-1.49,-1.57,6.31,16.09,-8.29,...,Communication Services,Internet Content & Information,United States,Alphabet Inc. offers various products and plat...,0.48,19.343750,19.343750,39809440,32413440,-0.035128
3,ABBV,185.49,214.68,-13.60,-0.46,0.01,-1.94,-0.89,15.34,5.31,...,Healthcare,Drug Manufacturers - General,United States,"AbbVie Inc., a research-based biopharmaceutica...",3.54,79.269230,15.291839,7701938,4394080,0.088620
2,ABT,132.41,139.57,-5.13,8.03,0.10,-0.74,-0.49,8.10,17.88,...,Healthcare,Medical Devices,United States,"Abbott Laboratories, together with its subsidi...",1.78,17.173801,25.660854,6692629,4782250,0.252459
26,AXP,296.42,324.79,-8.74,4.63,1.23,-1.70,7.64,20.98,-0.11,...,Financial Services,Credit Services,United States,"American Express Company, together with its su...",1.11,20.699722,19.604498,3185351,2358770,0.286657
6,AMD,126.79,211.38,-40.02,2.12,-0.24,4.16,28.56,15.21,5.11,...,Technology,Semiconductors,United States,"Advanced Micro Devices, Inc. operates as a sem...",NaN,92.547450,24.860785,43686245,51216520,-0.213608
47,T,27.66,28.42,-2.67,14.30,0.04,-0.72,-1.78,9.63,24.00,...,Communication Services,Telecom Services,United States,AT&T Inc. provides telecommunications and tech...,4.01,16.969326,12.348214,37241861,28881250,0.503261
4,ACN,306.38,396.28,-22.69,-9.65,-1.81,-3.23,0.85,8.57,-11.36,...,Technology,Information Technology Services,Ireland,Accenture plc provides strategy and consulting...,1.93,25.258038,21.775410,3365711,2795730,-0.008415


# Save to CSV

In [22]:
df_enriched_stock_data.to_csv('/Users/ani/Projects/6_stock_portfolio_recommendation/data/enriched_stock_data.csv', index=False)
print("Enriched stock data saved to 'data/enriched_stock_data.csv'")

Enriched stock data saved to 'data/enriched_stock_data.csv'
